## 52단계: GPU 지원

> 딥러닝으로 하는 계산은 '행렬의 곱'이 대부분을 차지합니다. 그런데 행렬의 곱은 곱셉과 덧셈으로 구성되어 있어서 병렬로 계산하는 게 가능하고, 병렬 계산에는 CPU보다 GPU가 훨씬 뛰어 납니다. 그래서 이번 단계에서는 DeZero를 GPU에서 구동하기 위한 구조를 만들 것입니다.

### 52.1 쿠파이 설치 및 사용 방법

```bash
$ pip install cupy
```

In [1]:
import numpy as np
import cupy as cp

np_x = np.arange(6).reshape(2, 3)
print(np_x)
cp_x = cp.arange(6).reshape(2, 3)
print(cp_x)

np_y = np_x.sum(axis=1)
print(np_y)
cp_y = cp_x.sum(axis=1)
print(cp_y)

[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]
[ 3 12]
[ 3 12]


주의: numpy와 cupy의 API는 거의 동일하지만 완전히 같지는 않다.

In [2]:
# numpy -> cupy
n = np.array([1, 2, 3])
c = cp.asarray(n)

print(isinstance(c, cp.ndarray))

# cupy -> numpy
c = cp.array([1, 2, 3])
n = cp.asnumpy(c)
print(isinstance(n, np.ndarray))

True
True


In [3]:
# x가 넘파이 배열인 경우
x = np.array([1, 2, 3])
xp = cp.get_array_module(x)
print(xp == np)

# x가 쿠파이 배열인 경우
x = cp.array([1, 2, 3])
xp = cp.get_array_module(x)
print(xp == cp)

True
True


### 52.2 쿠다 모듈

In [4]:
# dezero/cuda.py

import numpy as np
gpu_enable = True
try:
    import cupy as cp
    cupy = cp
except ImportError:
    gpu_enable = False
from dezero import Variable


# cupy는 필수가 아니므로 설치되지 않은 경우에도 동작하도록 설계
def get_array_module(x):
    if isinstance(x, Variable):
        x = x.data
    
    if not gpu_enable:
        return np
    xp = cp.get_array_module(x)
    return xp

def as_numpy(x):
    if isinstance(x, Variable):
        x = x.data
    
    if np.isscalar(x):
        return np.array(x)
    elif isinstance(x, np.ndarray):
        return x
    return cp.asnumpy(x)

def as_cupy(x):
    if isinstance(x, Variable):
        x = x.data
    
    if not gpu_enable:
        raise Exception('CuPy를 로드할 수 없습니다. CuPy를 먼저 설치해주세요.')
    return cp.asarray(x)

### 52.3 Variable/Layer/DataLoader 클래스 추가 구현

In [5]:
# dezero/core.py

import dezero

...
try:  # 동적으로 array_types 설정
    import cupy
    array_types = (np.ndarray, cupy.ndarray)
except ImportError:
    array_types = (np.ndarray,)

class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, array_types):
                raise TypeError('{} is not supported'.format(type(data)))
        ...
    
    def backward(self, retain_grad = False, create_graph=False):
        if self.grad is None:
            xp = dezero.cuda.get_array_module(self.data)  # 데이터 타입에 따라 모듈 선택
            self.grad = Variable(xp.ones_like(self.data))
        ...
    
    ...
    def to_cpu(self):  # 데이터를 cpu로 전송
        if self.data is not None:
            self.data = dezero.cuda.as_numpy(self.data)
            
    def to_gpu(self):  # 데이터를 gpu로 전송
        if self.data is not None:
            self.data = dezero.cuda.as_cupy(self.data)

In [6]:
# dezero/layers.py

class Layer:
    ...
    
    def to_cpu(self):
        for param in self.params():
            param.to_cpu()
            
    def to_gpu(self):
        for param in self.params():
            param.to_gpu()

In [7]:
# dezero/dataloaders.py

...
from dezero import cuda

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        ...
        self.gpu = gpu
        
        self.reset()
    
    def __next__(self):
        ...
        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[0] for example in batch])
        
        self.iteration += 1
        
        return x, t
    
    def to_cpu(self):
        self.gpu = False

    def to_gpu(self):
        self.gpu = True

### 52.4 함수 추가 구현

아래는 Sin 클래스의 수정 예시이다. `functions.py`, `optimizers.py`, `layers.py`에도 필요한 부분에 전부 적용해줘야 한다.

In [8]:
# dezero/functions.py

from dezero import Function

class Sin(Function):
    def forward(self, x):
        xp = cuda.get_array_module(x)  # 동적으로 모듈 선택
        y = xp.sin(x)
        return y
    ...

사칙연산도 수정해준다.

In [9]:
# dezero/core.py

def as_array(x, array_module=np):
    if np.isscalar(x):
        return array_module.array(x)
    return x

def add(x0, x1):
    x1 = as_array(x1, dezero.cuda.get_array_module(x0.data))
    return Add()(x0, x1)

def mul(x0, x1):
    x1 = as_array(x1, dezero.cuda.get_array_module(x0.data))
    return Mul()(x0, x1)

# sub, rsub, div, rdiv 전부 수정

### 52.5 GPU로 MNIST 학습하기

In [10]:
import time
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP


max_epoch = 5
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

# GPU mode
if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9166, time: 7.2556[sec]
epoch: 2, loss: 1.2864, time: 5.1546[sec]
epoch: 3, loss: 0.9261, time: 5.1708[sec]
epoch: 4, loss: 0.7403, time: 5.3612[sec]
epoch: 5, loss: 0.6351, time: 5.4565[sec]


In [11]:
train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

# GPU mode -> CPU mode
# if dezero.cuda.gpu_enable:
#     train_loader.to_gpu()
#     model.to_gpu()

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9129, time: 23.8272[sec]
epoch: 2, loss: 1.2801, time: 26.7616[sec]
epoch: 3, loss: 0.9207, time: 17.0228[sec]
epoch: 4, loss: 0.7373, time: 17.0101[sec]
epoch: 5, loss: 0.6334, time: 21.7303[sec]
